<!-- ---
title: "Stochastic Gradient Descent"
format: 
  revealjs:
    theme: custom.scss
    css: custom.css
    width: 1920
    height: 1080
    menu:
      side: right
      width: wide
    template-partials:
      - title-slide.html
    slide-number: c/t
    logo: "eclipse_logo_small.png"
    highlight-style: a11y
    incremental: false
    background-transition: fade
    footer: "©Philipp Pelz - FAU Erlangen-Nürnberg - Data Science for Electron Microscopy"
execute:
  eval: true
  echo: true
--- -->

# Stochastic Gradient Descent

- Previously used SGD without detailed explanation
- Now diving deeper into its principles
- Building on gradient descent fundamentals
- Understanding why and how it works


In [ ]:
#| label: setup4
#| code-fold: true
%matplotlib inline
import d2l
import math
import torch

## Stochastic Gradient Updates

### Objective Function

- Training dataset with $n$ examples
- Loss function $f_i(\mathbf{x})$ for example $i$
- Overall objective:
  $$f(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n f_i(\mathbf{x})$$
- Full gradient:
  $$\nabla f(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n \nabla f_i(\mathbf{x})$$

---

### Computational Cost

- Gradient descent: $\mathcal{O}(n)$ per iteration
- SGD: $\mathcal{O}(1)$ per iteration
- Update rule:
  $$\mathbf{x} \leftarrow \mathbf{x} - \eta \nabla f_i(\mathbf{x})$$
- Unbiased estimate:
  $$\mathbb{E}_i \nabla f_i(\mathbf{x}) = \nabla f(\mathbf{x})$$

### Implementation


In [ ]:
#| label: objective-fn
def f(x1, x2):  # Objective function
    return x1 ** 2 + 2 * x2 ** 2

def f_grad(x1, x2):  # Gradient of the objective function
    return 2 * x1, 4 * x2

---


In [ ]:
#| label: sgd-implementation
def sgd(x1, x2, s1, s2, f_grad):
    g1, g2 = f_grad(x1, x2)
    # Simulate noisy gradient
    g1 += torch.normal(0.0, 1, (1,)).item()
    g2 += torch.normal(0.0, 1, (1,)).item()
    eta_t = eta * lr()
    return (x1 - eta_t * g1, x2 - eta_t * g2, 0, 0)

In [ ]:
#| label: constant-lr
def constant_lr():
    return 1

eta = 0.1
lr = constant_lr  # Constant learning rate
d2l.show_trace_2d(f, d2l.train_2d(sgd, steps=50, f_grad=f_grad))

## Dynamic Learning Rate

### Learning Rate Strategies

- Piecewise constant: $\eta(t) = \eta_i \textrm{ if } t_i \leq t \leq t_{i+1}$
- Exponential decay: $\eta(t) = \eta_0 \cdot e^{-\lambda t}$
- Polynomial decay: $\eta(t) = \eta_0 \cdot (\beta t + 1)^{-\alpha}$

### Exponential Decay Implementation


In [ ]:
#| label: exponential-lr
def exponential_lr():
    global t
    t += 1
    return math.exp(-0.1 * t)

t = 1
lr = exponential_lr
d2l.show_trace_2d(f, d2l.train_2d(sgd, steps=1000, f_grad=f_grad))

---


### Polynomial Decay Implementation


In [ ]:
#| label: polynomial-lr
def polynomial_lr():
    global t
    t += 1
    return (1 + 0.1 * t) ** (-0.5)

t = 1
lr = polynomial_lr
d2l.show_trace_2d(f, d2l.train_2d(sgd, steps=50, f_grad=f_grad))

## Stochastic Gradients and Finite Samples

### Sampling Strategies

- With replacement:
  - Probability of choosing element: $1 - e^{-1} \approx 0.63$
  - Increased variance
  - Decreased data efficiency
- Without replacement:
  - Better variance properties
  - More efficient data usage
  - Default choice in practice

## Summary

- Key points:
  - SGD reduces computational cost to $\mathcal{O}(1)$
  - Learning rate scheduling is crucial
  - Convergence guarantees for convex problems
  - Sampling without replacement preferred
- Practical considerations:
  - Dynamic learning rates
  - Trade-offs in sampling strategies
  - Nonconvex optimization challenges

## Exercises

1. Experiment with learning rate schedules
2. Analyze noise in gradient updates
3. Compare sampling strategies
4. Investigate gradient coordinate scaling
5. Study local minima in nonconvex functions 